# Social RL: Learning Through Interaction

This notebook demonstrates the Social RL architecture with **vLLM + Qwen** - a novel approach where agents learn from social feedback without explicit reward functions.

| Traditional RL | Social RL |
|---------------|----------|
| Environment | Other agents + theoretical constraints |
| State | Round context + concept manifestations |
| Action | Agent utterance/response |
| Reward | Social feedback (engagement, alignment, contribution) |
| Policy | PRAR process schemas |

## 1. Environment Setup

In [ ]:
# Clear GPU and prepare environment
import gc
import torch
import os

print("Stopping old processes...")
!pkill -f vllm

print("Freeing GPU memory...")
gc.collect()
torch.cuda.empty_cache()

!nvidia-smi

In [ ]:
# Install dependencies
!pip install -q "vllm==0.6.6" openai

## 2. Launch vLLM Server

In [ ]:
import os
import time

os.environ['VLLM_WORKER_MULTIPROC_METHOD'] = 'spawn'

print("Starting vLLM server...")
!nohup python -m vllm.entrypoints.openai.api_server \
  --model Qwen/Qwen2.5-7B-Instruct \
  --dtype bfloat16 \
  --port 8000 \
  --host 0.0.0.0 \
  > vllm.log 2>&1 &

print("Waiting for server to start (30s)...")
time.sleep(30)
!tail -n 20 vllm.log

In [ ]:
# Verify server is running
!curl -s http://127.0.0.1:8000/v1/models | python -m json.tool

## 3. Clone Repository

In [ ]:
import os
import sys

if not os.path.exists('/content/Socratic-RCM'):
    !git clone https://github.com/Baglecake/Socratic-RCM.git /content/Socratic-RCM
else:
    !cd /content/Socratic-RCM && git pull

%cd /content/Socratic-RCM

# Add paths
sys.path.insert(0, '/content/Socratic-RCM/social_rl')
sys.path.insert(0, '/content/Socratic-RCM/agents')
sys.path.insert(0, '/content/Socratic-RCM/local_rcm')

# Verify social_rl exists
!ls -la social_rl/

## 4. Create LLM Client

In [ ]:
from openai import OpenAI

class VLLMClient:
    """LLM client using vLLM OpenAI-compatible API."""
    
    def __init__(self, base_url="http://127.0.0.1:8000/v1", model="Qwen/Qwen2.5-7B-Instruct"):
        self.client = OpenAI(api_key="not-needed", base_url=base_url)
        self.model = model
        self.call_count = 0
    
    def send_message(self, system_prompt: str, user_message: str, temperature: float = 0.7) -> str:
        """Send message to LLM and return response."""
        self.call_count += 1
        
        # Add language enforcement for Qwen
        system_prompt += "\n\nIMPORTANT: Always respond in English only."
        
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_message}
            ],
            temperature=temperature,
            max_tokens=512
        )
        return response.choices[0].message.content

# Test connection
llm = VLLMClient()
test_response = llm.send_message(
    "You are a helpful assistant.",
    "Say 'Social RL is ready!' in exactly those words."
)
print(f"LLM Response: {test_response}")

## 5. Load Canvas & Create Social RL Runner

In [ ]:
import json
from runner import SocialRLRunner, SocialRLConfig

# Load baseline canvas
state_path = '/content/Socratic-RCM/prar/outputs/2025-11-23_baseline_full_qwen/state.json'
with open(state_path, 'r') as f:
    canvas = json.load(f)['canvas']

print(f"Framework: {canvas['project'].get('theoretical_option_label')}")
print(f"Agents: {[a['identifier'] for a in canvas['agents']]}")
print(f"Rounds: {len(canvas['rounds'])}")

In [ ]:
# Create Social RL Runner with vLLM
config = SocialRLConfig(
    manifestation_mode="progressive",  # Options: static, progressive, reactive, adaptive
    use_prar_cues=True,
    prar_intensity="medium",
    use_coach_validation=False,  # Set True to enable Coach validation
    verbose=True
)

runner = SocialRLRunner(canvas, llm, config)
print("\nSocialRLRunner created with vLLM backend!")

## 6. Execute Social RL Round 1

In [ ]:
# Execute Round 1
result1 = runner.execute_round(round_number=1, max_turns=9)  # 3 agents x 3 turns each
print(f"\nRound 1 complete: {len(result1.messages)} messages")

In [ ]:
# View Round 1 Feedback
print("=" * 60)
print("ROUND 1 FEEDBACK ANALYSIS")
print("=" * 60)

for agent_id, fb in result1.feedback.items():
    print(f"\n{agent_id}:")
    print(f"  Engagement: {fb.engagement:.2f}")
    print(f"  Theoretical Alignment: {fb.theoretical_alignment:.2f}")
    print(f"  Contribution Value: {fb.contribution_value:.2f}")
    print(f"  Concepts Embodied: {list(set(fb.concepts_embodied))}")

## 7. Execute Round 2 (With Policy Adaptation)

In [ ]:
# Execute Round 2 - policies adapt based on Round 1 feedback
print("\n" + "=" * 60)
print("ROUND 2 - With policy adaptation from Round 1 feedback")
print("=" * 60)

result2 = runner.execute_round(round_number=2, max_turns=9)
print(f"\nRound 2 complete: {len(result2.messages)} messages")

if result2.policy_adaptations:
    print("\nPolicy Adaptations Made:")
    for adapt in result2.policy_adaptations:
        print(f"  {adapt['agent_id']}: {adapt['deltas']}")

In [ ]:
# Compare feedback across rounds
print("=" * 60)
print("FEEDBACK DELTA (Round 1 -> Round 2)")
print("=" * 60)

comparison = runner.feedback_extractor.compare_rounds(1, 2)
for agent_id, deltas in comparison.items():
    print(f"\n{agent_id}:")
    for signal, delta in deltas.items():
        direction = "+" if delta > 0 else ""
        print(f"  {signal}: {direction}{delta:.3f}")

## 8. Full Simulation Report

In [ ]:
# Generate comprehensive report
print(runner.generate_report())

In [ ]:
# Save results
output_dir = '/content/Socratic-RCM/outputs/social_rl_run'
runner.save_results(output_dir)
print(f"\nResults saved to: {output_dir}")
!ls -la {output_dir}/

## 9. Examine Dynamic Context Injection

In [ ]:
# Show how context evolved for one agent across turns
alice_messages = [m for m in result1.messages if m.agent_id == "Worker+Alice"]

print("=" * 60)
print("ALICE'S CONTEXT EVOLUTION (Round 1)")
print("=" * 60)

for msg in alice_messages:
    print(f"\n--- Turn {msg.turn_number} ---")
    print(f"Response: {msg.content[:200]}...")
    if msg.prar_cue_used:
        print(f"\nPRAR Cue: {msg.prar_cue_used[:150]}...")
    if msg.feedback_snapshot:
        print(f"Feedback at turn: {msg.feedback_snapshot}")

## Key Insights

### What Makes This "Social RL"?

1. **No Explicit Reward Function**: Learning signals emerge from interaction
   - Being referenced = engagement signal
   - Embodying concepts = alignment signal
   - Inclusion in synthesis = contribution signal

2. **Process Retrieval as Policy**: PRAR guides HOW to reason, not WHAT to say
   - Policies adapt based on feedback deltas
   - No model weight updates needed

3. **Dynamic Context Injection**: Each turn gets fresh manifestations
   - Intensity scales: low → medium → high through the round
   - Context reacts to conversation dynamics

4. **Theoretical Grounding**: Framework prevents drift
   - Concepts constrain the possibility space
   - Analyst coding reinforces alignment